In [1]:
from info_nas.datasets.networks.pretrained import pretrain_network_cifar
from info_nas.datasets.networks.utils import load_nasbench
from nasbench_pytorch.datasets.cifar10 import prepare_dataset

In [3]:
import os

os.listdir('..')

['.git',
 '.gitignore',
 '.idea',
 'data',
 'info_nas',
 'info_nas.egg-info',
 'LICENSE',
 'notebooks',
 'README.md',
 'setup.py']

In [5]:
seed = 42
batch_size = 128
num_workers = 8

dataset = prepare_dataset(batch_size, root='../data/cifar/', validation_size=1000, random_state=seed,
                          num_workers=num_workers)
train, n_train, val, n_val, test, n_test = dataset


--- Preparing CIFAR10 Data ---
Files already downloaded and verified
Files already downloaded and verified
--- CIFAR10 Data Prepared ---



In [5]:
print('train: ', n_train)
print('validation: ', n_val)
print('test: ', n_test)

train:  49000
validation:  1000
test:  10000


In [4]:
nasbench_path = '../data/nasbench_only108.tfrecord'

nb = load_nasbench(nasbench_path)

Loading dataset from file... This may take a few minutes...
Loaded dataset in 33 seconds


In [7]:
len(nb)

423624

In [6]:
nets = nb[:50]

In [ ]:
nets[0]

In [7]:
from nasbench_pytorch.model import Network

num_labels = 10

def to_model(ops, mat):
    return Network((mat, ops), num_labels)

net_dict = {v[0]: to_model(v[1], v[2]) for v in nets}

In [8]:
from info_nas.datasets.config import load_json_cfg

config = load_json_cfg('../info_nas/configs/pretrain_config.json')
config

{'cifar-10': {'batch_size': 256, 'validation_size': 1000, 'num_workers': 8},
 'nb_dataset': {'test_size': 0.1},
 'pretrain': {'num_epochs': 12, 'optimizer': 'SGD'}}

In [9]:
import torch
from info_nas.datasets.networks.utils import save_trained_net
from datetime import datetime

torch.backends.cudnn.benchmark = True
device = torch.device('cuda')

for net_hash, net in net_dict.items():
    now = datetime.now()
    print(now.strftime("%d/%m/%Y %H:%M:%S\n--------------------"))
    
    # train net and save it
    net = net.to(device)
    net, metrics = pretrain_network_cifar(net, train, val, test, device=device,
                                          print_frequency=50, **config['pretrain'])
    
    save_trained_net(net_hash, net, info=metrics, net_args=[num_labels])

22/06/2021 15:26:01
--------------------
Epoch=0/12 Batch=1/383 | Loss=2.309, Acc=0.133(17/128)
Epoch=0/12 Batch=51/383 | Loss=2.089, Acc=0.218(1421/6528)
Epoch=0/12 Batch=101/383 | Loss=1.936, Acc=0.274(3543/12928)
Epoch=0/12 Batch=151/383 | Loss=1.840, Acc=0.314(6063/19328)
Epoch=0/12 Batch=201/383 | Loss=1.764, Acc=0.345(8878/25728)
Epoch=0/12 Batch=251/383 | Loss=1.690, Acc=0.375(12056/32128)
Epoch=0/12 Batch=301/383 | Loss=1.632, Acc=0.400(15408/38528)
Epoch=0/12 Batch=351/383 | Loss=1.586, Acc=0.419(18827/44928)
Testing: Loss=1.228, Acc=0.548(548/1000)
--------------------
Epoch=1/12 Batch=1/383 | Loss=1.060, Acc=0.586(75/128)
Epoch=1/12 Batch=51/383 | Loss=1.129, Acc=0.593(3870/6528)
Epoch=1/12 Batch=101/383 | Loss=1.117, Acc=0.598(7733/12928)
Epoch=1/12 Batch=151/383 | Loss=1.109, Acc=0.601(11609/19328)
Epoch=1/12 Batch=201/383 | Loss=1.092, Acc=0.608(15639/25728)
Epoch=1/12 Batch=251/383 | Loss=1.070, Acc=0.617(19837/32128)
Epoch=1/12 Batch=301/383 | Loss=1.049, Acc=0.625(2408

KeyboardInterrupt: 

In [ ]:
#jen 10 epochs for testing (108 trvá)